In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import config
import utility.df
import utility.iolib
import utility.plotlib

#env = sys.argv[1] if len(sys.argv) > 2 else "dev"

In [2]:
# Setup configuration
cfg = config.ResearchConfig
time_format = cfg.CSV_TIME_FORMAT
date_format = cfg.CSV_DATE_FORMAT
cutoff_date = pd.to_datetime(cfg.CUTOFF_DATE, format=cfg.CSV_DATE_FORMAT)

# Retrieve data
df_subspt, df_lesson, df_incomp, df_crclum = utility.iolib.retrieve_data(cfg)
print("Complete loading data for subscription and lesson history!")

Complete loading data for subscription and lesson history!


In [3]:
pupil_list_mon = df_subspt[df_subspt.subscription_type==cfg.TYPE_MONTHLY].pupilId.unique()
pupil_list_ann = df_subspt[df_subspt.subscription_type==cfg.TYPE_ANNUAL].pupilId.unique()
print('Number of monthly subscribers = {}'.format(len(pupil_list_mon)))
print('Number of annual subscribers = {}'.format(len(pupil_list_ann)))

Number of monthly subscribers = 1398
Number of annual subscribers = 1015


In [4]:
df_usage = utility.df.customer_usage(df_subspt, df_lesson, df_incomp, cfg)

By the cutoff date 2018-04-20, there are 973 active subscriptions.
These subscribers shall be removed from the analysis because we have no evidence to know the lifetime of their subscriptions

In the first month of dataset, there are 110 renewal or new subscriptions.
These subscribers shall be removed from the analysis because we have no evidence to show if they renewed or newly joined. 

In summary, there are 1075 / 2324 subscribers are removed from the dataset in the analysis. 

Start assigning customer month for complete lesson table.
Complete processing 261 / 5221 records in the subscription table.
Complete processing 522 / 5221 records in the subscription table.
Complete processing 783 / 5221 records in the subscription table.
Complete processing 1044 / 5221 records in the subscription table.
Complete processing 1305 / 5221 records in the subscription table.
Complete processing 1566 / 5221 records in the subscription table.
Complete processing 1827 / 5221 records in the subscripti

In [5]:
df_usage

customer_month  time_taken_complete  time_taken_incomplete
pupilId date                                                                  
988318  2016-07-16             1.0                451.0                    0.0
        2016-07-18             1.0               2012.0                    0.0
        2016-07-20             1.0               3640.0                  230.0
        2016-07-21             1.0                  0.0                  124.0
        2016-07-22             1.0               3270.0                 1536.0
        2016-07-24             1.0                472.0                   19.0
        2016-07-25             1.0               1785.0                    0.0
        2016-07-26             1.0                755.0                    0.0
        2016-07-27             1.0               1195.0                    0.0
        2016-07-30             1.0               1435.0                    0.0
        2016-08-01             1.0               1361.0                    0.0
        2016-08-02             1.0                869.0                    0.0
        2016-08-03             1.0               1115.0                  960.0
        2016-08-04             1.0                371.0                  639.0
        2016-08-05             1.0               2014.0                    0.0
        2016-08-06             1.0                686.0                    0.0
        2016-08-07             1.0               1925.0                    0.0
        2016-08-17             2.0               1664.0                    0.0
        2016-08-18             2.0               2151.0                    0.0
        2016-08-19             2.0               2951.0                  944.0
        2016-08-22             2.0               1765.0                    0.0
        2016-08-23             2.0                224.0                  826.0
        2016-08-25             2.0               2009.0                  311.0
        2016-08-29             2.0               1168.0                  696.0
        2016-08-31             2.0               1726.0                    0.0
994953  2016-03-26             1.0                916.0                    0.0
        2016-03-28             1.0                486.0                    0.0
        2016-03-29             1.0                781.0                    0.0
        2016-03-30             1.0                606.0                    0.0
        2016-04-01             1.0               1262.0                   44.0
...                            ...                  ...                    ...
1281418 2018-04-10             2.0               1751.0                    0.0
1284479 2018-02-25             1.0               3003.0                  259.0
        2018-02-26             1.0                  0.0                  507.0
        2018-03-01             1.0               2122.0                    0.0
        2018-03-10             1.0               1028.0                   14.0
1288697 2018-03-18             1.0               1922.0                  644.0
        2018-03-19             1.0               2500.0                    0.0
        2018-03-20             1.0               1445.0                  172.0
        2018-03-22             1.0               2204.0                  829.0
        2018-03-27             1.0               2967.0                    0.0
        2018-03-29             1.0               2520.0                   12.0
        2018-03-30             1.0                898.0                   14.0
        2018-03-31             1.0               1782.0                    0.0
        2018-04-02             1.0               1049.0                   13.0
        2018-04-03             1.0               4688.0                    7.0
        2018-04-06             1.0               3050.0                  705.0
        2018-04-07             1.0               3812.0                    0.0
        2018-04-08             1.0               2032.0               